In [1]:
def rule(name: str, match: str, build: str, where: str):
    return rule(name, match, build) + f"where {where}"

def rule(name: str, match: str, build: str):
    return f"{name}: {match} -> {build}"

In [2]:
MODULE_NAME = "benchnullary"
RULE_NAME = "R"
SORT_NAME = "S"
CONS_NAME = "Constr{}"
BUILD = "Constr{}"
NUM_REPEATS = 1000

PROGRAM_SIZE_LOWER = 100
PROGRAM_SIZE_UPPER = 500
PROGRAM_SIZE_STEP = 100

PROGRAM_OUTPUT_DIR = "generated-benchmarks"

In [15]:
import os

def PROGRAM(input_size: int, num_constructors: int):
    constructor_ids = range(1, num_constructors + 1)
    module_name = MODULE_NAME + str(input_size)
    return {
        "module_name": module_name + ".str2", 
        "contents": f"""
module {module_name}

imports libstratego-lib

signature
  sorts {SORT_NAME}
  constructors
{os.linesep.join([f"    {CONS_NAME.format(i)} : {SORT_NAME}" for i in constructor_ids])}

strategies

  main = <repeat({RULE_NAME}|{NUM_REPEATS})> {CONS_NAME.format(input_size)}()


rules

{os.linesep.join([f"  {rule(RULE_NAME, CONS_NAME.format(i) + '()', CONS_NAME.format(i) + '()')}" for i in constructor_ids])}

"""}

print(PROGRAM(2, 4)["contents"])


module benchnullary2

imports libstratego-lib

signature
  sorts S
  constructors
    Constr1 : S
    Constr2 : S
    Constr3 : S
    Constr4 : S

strategies

  main = <repeat(R|1000)> Constr2()


rules

  R: Constr1() -> Constr1()
  R: Constr2() -> Constr2()
  R: Constr3() -> Constr3()
  R: Constr4() -> Constr4()




In [16]:
from os import path

for input_size in range(PROGRAM_SIZE_LOWER, PROGRAM_SIZE_UPPER + 1, PROGRAM_SIZE_STEP):
    prog = PROGRAM(input_size, PROGRAM_SIZE_UPPER)
    
    filename = path.join(PROGRAM_OUTPUT_DIR, prog['module_name'])
    contents = prog["contents"]
    
    os.makedirs(path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as prog_file:
        prog_file.write(contents)

In [13]:
import os

def TIL_add_program(input_size: int):
    module_name = "add" + str(input_size) + ".til"
    return {
        "module_name": module_name, 
        "contents": f"""
begin
var n0;
n0 := readint();
{os.linesep.join([f"var n{i+1}; n{i+1} := n{i} + 1;" for i in range(input_size)])}
write(n{input_size});
end
"""}

print(TIL_add_program(10)["contents"])


begin
var n0;
n0 := readint();
var n1; n1 := n0 + 1;
var n2; n2 := n1 + 1;
var n3; n3 := n2 + 1;
var n4; n4 := n3 + 1;
var n5; n5 := n4 + 1;
var n6; n6 := n5 + 1;
var n7; n7 := n6 + 1;
var n8; n8 := n7 + 1;
var n9; n9 := n8 + 1;
var n10; n10 := n9 + 1;
write(n10);
end



In [17]:
from os import path

for input_size in range(PROGRAM_SIZE_LOWER, (2*PROGRAM_SIZE_UPPER) + 1, PROGRAM_SIZE_STEP):
    prog = TIL_add_program(input_size)
    
    filename = path.join(PROGRAM_OUTPUT_DIR, prog['module_name'])
    contents = prog["contents"]
    
    os.makedirs(path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as prog_file:
        prog_file.write(contents)